## Computing phases in a ternary system
This method is based on the [paper](https://pubs.rsc.org/en/content/articlelanding/2019/sm/c8sm02045k#!divAbstract) which uses a convex envelope method but identifies the potential phases using a Laplacian based approach

In [46]:
%config InlineBackend.figure_format = 'svg'
import mpltern
from matplotlib import rc
rc('text', usetex=True)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib widget

import pdb
import numpy as np

import sys
if '../' not in sys.path:
    sys.path.append('../')

from solvers.utils import get_data

import warnings
warnings.filterwarnings("ignore")

In [47]:
# have all the helpers functions here

# make a general grid
def makegridnd(num_points,dim):
    x = np.meshgrid(*[np.linspace(0.001, 1,num_points) for d in range(dim)])
    mesh = np.asarray(x)
    total = np.sum(mesh,axis=0)
    plane_mesh = mesh[:,np.isclose(total,1.0,atol=1e-2)]
    
    return plane_mesh

def utri2mat(utri,dim):
    inds = np.triu_indices(dim,1)
    ret = np.zeros((dim, dim))
    ret[inds] = utri
    ret.T[inds] = utri
    return ret

from scipy.spatial.distance import pdist, euclidean, squareform
from scipy.sparse import csr_matrix
import pandas as pd
from scipy.sparse.csgraph import connected_components

def label_triangle(triangle, coords, thresh):
    tri_coords = [coords[:,x] for x in triangle]
    dist = squareform(pdist(tri_coords,'euclidean'))
    adjacency = dist<thresh
    adjacency =  adjacency.astype(int)  
    graph = csr_matrix(adjacency)
    n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
    
    return n_components

def get_phase_diagram(simplices, coords, thresh):
    num_comps = [label_triangle(triangle, coords, thresh) for triangle in simplices]
    
    return num_comps

def get_faces(v):
    # generate list of sides' polygons of our pyramid
    verts = [ [v[0],v[1],v[3]], [v[1],v[2],v[3]],
     [v[0],v[2],v[3]], [v[0],v[1],v[2]]]
    return verts

def from4d23d(fourd_coords):
    x,y,z,w = fourd_coords
    u = y+0.5*(z+w)
    v = np.sqrt(3)*(z/2 + w/6) 
    w = np.sqrt(6)*(w/3)
    
    return [u,v,w] 

In [48]:
num_points = 40
dim = 4
grid = makegridnd(num_points,dim)

print(grid.shape)
                         

(4, 11480)


In [49]:
# get your pure points
di = np.diag_indices(dim)
pure_points = 0.001*np.ones((dim,dim))
pure_points[di]=1
print(pure_points)
verts = pure_points[:,:3].tolist()
print(verts)

[[1.    0.001 0.001 0.001]
 [0.001 1.    0.001 0.001]
 [0.001 0.001 1.    0.001]
 [0.001 0.001 0.001 1.   ]]
[[1.0, 0.001, 0.001], [0.001, 1.0, 0.001], [0.001, 0.001, 1.0], [0.001, 0.001, 0.001]]


In [50]:
M = np.ones(dim) 
chi = 3.10*np.ones(int(0.5*dim*(dim-1)))
CHI = utri2mat(chi, dim)
print(CHI)

[[0.  3.1 3.1 3.1]
 [3.1 0.  3.1 3.1]
 [3.1 3.1 0.  3.1]
 [3.1 3.1 3.1 0. ]]


In [51]:
from solvers import helpers

gmix = lambda x: helpers.flory_huggins(x, M, CHI,beta=1e-4)
energy = []
reparam_coords = []
for i in range(grid.shape[1]):
    energy.append(gmix(grid[:,i]))
    reparam_coords.append(from4d23d(grid[:,i]))

In [52]:
# Plot a convex triangulation in the ternary space
from solvers.helpers import get_ternary_coords
from scipy.spatial import ConvexHull

reparam_coords = np.asarray(reparam_coords)
points = np.concatenate((reparam_coords,np.asarray(energy).reshape(-1,1)),axis=1)
hull = ConvexHull(points)

In [53]:
simplices = []
for simplex in hull.simplices:
    point_class = np.sum(np.isclose(grid[:,simplex],0.001),axis=1)
    point_class = np.unique(dim - point_class)
    if (point_class==1).any():
        pass
    else:
        simplices.append(simplex)

In [55]:
thresh = 5*euclidean(reparam_coords[:,0],reparam_coords[:,1])
num_comps = get_phase_diagram(simplices,reparam_coords.T, thresh)

In [58]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# vertices of a pyramid
v = np.array([[0, 0, 0], [1, 0, 0], [1/2,np.sqrt(3)/2,0],  [1/2,np.sqrt(3)/6,np.sqrt(6)/3]])
ax.scatter3D(v[:, 0], v[:, 1], v[:, 2],color='black')

verts = get_faces(v)
# plot sides
ax.add_collection3d(Poly3DCollection(verts, facecolors='black', linewidths=0.5, edgecolors='black', alpha=.05))
#ax.scatter3D(coords[:, 0], coords[:, 1], coords[:, 2],color='black')
for i,simplex in zip(num_comps,simplices):
    vertices = [grid[:,x] for x in simplex]
    v = np.asarray([from4d23d(vertex) for vertex in vertices])
    if np.all(np.asarray(vertices)[:,3]<1.0):
        verts = get_faces(v)
        if i==1:
            ax.add_collection3d(Poly3DCollection(verts, facecolors='tab:red', edgecolors=None))
        if i==2:
            ax.add_collection3d(Poly3DCollection(verts, facecolors='tab:olive', edgecolors=None))
        if i==3:
            ax.add_collection3d(Poly3DCollection(verts, facecolors='tab:cyan', edgecolors=None))
#             if i==4:
#                 ax.add_collection3d(Poly3DCollection(verts, facecolors='tab:purple', edgecolors=None))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …